In [ ]:
# @title 1. Installing Dependencies
import os
import sys

# Cleanup
print("Clearing environment...")
os.system("pip uninstall -y numpy")
os.system("pip uninstall -y opencv-python")
os.system("pip uninstall -y opencv-python-headless")
os.system("pip uninstall -y opencv-contrib-python")

# FTF
print("Installing heavy libraries...")
!pip install -q markdown xhtml2pdf
!pip install paddlepaddle-gpu==2.6.1
!pip install paddleocr==2.7.3
!apt-get update -qq && apt-get install -y -qq libgl1-mesa-glx
!pip install accelerate bitsandbytes
!pip install fastapi uvicorn python-multipart pyngrok nest-asyncio
!pip install "langchain==0.1.20" "langchain-community==0.0.38"
!pip install "transformers==4.41.2" --force-reinstall


print("Locking versions together...")
!pip install "numpy==1.26.4" "opencv-python-headless==4.8.0.74" "opencv-contrib-python-headless==4.8.0.74" --force-reinstall

print("\n INSTALLATION COMPLETE.")
print"restart now")

Clearing environment...


In [ ]:
# @title 2. Logic Engine
import torch
from paddleocr import PaddleOCR
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from PIL import Image, ImageEnhance
import numpy as np
import io
import warnings
import sys
import subprocess
import gc

# Memory Cleanup
if 'digitizer' in locals(): del digitizer
gc.collect()
torch.cuda.empty_cache()

warnings.filterwarnings("ignore")

# CUDA Check
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Uing device: {device}")

class NotesDigitizer:
    def __init__(self):
        print("⏳ Loading OCR...")
        self.ocr_engine = PaddleOCR(use_angle_cls=True, lang='en', show_log=False)

        print("⏳ Loading LLM ...")
        model_id = "microsoft/Phi-3-mini-4k-instruct"
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)

        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            trust_remote_code=True,
            device_map="auto",
            attn_implementation="eager"
        )

        self.llm_pipe = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_new_tokens=2500,
            do_sample=True,
            temperature=0.1
        )
        print("Systems Online.")

    def preprocess_image(self, image_bytes):
        img = Image.open(io.BytesIO(image_bytes)).convert('RGB')
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(1.8)
        enhancer = ImageEnhance.Sharpness(img)
        img = enhancer.enhance(1.5)
        return np.array(img)

    def process_batch(self, image_files_bytes):
        """Process multiple images, extract text with OCR, and structure with LLM."""
        clean_pages = []
        total_pages = len(image_files_bytes)
        print(f"📚 Processing {total_pages} pages individually...")

        # OCR Phase
        for idx, img_bytes in enumerate(image_files_bytes):
            try:
                enhanced_image = self.preprocess_image(img_bytes)
                result = self.ocr_engine.ocr(enhanced_image, cls=True)
                page_text_lines = []
                if result and result[0]:
                    for line in result[0]:
                        text = line[1][0]
                        if "_____" in text:
                            continue
                        page_text_lines.append(text)

                # Format this page's text
                raw_text = "\n".join(page_text_lines)
                clean_pages.append(f"--- PAGE {idx+1} ---\n{raw_text}")
                print(f"   ✓ Page {idx+1} processed successfully")

            except Exception as e:
                print(f"   ✗ Error on Page {idx+1}: {str(e)[:100]}...")
                clean_pages.append(f"--- PAGE {idx+1} (FAILED) ---\n[OCR Error: {str(e)[:50]}...]")

        # Combine all pages for LLM processing
        full_context = "\n\n".join(clean_pages)

        # LLM Phase
        print("Structuring notes with LLM...")

        prompt = f"""<|user|>
You are a strict University Professor. You have been given a student's unordered, chaotic lecture notes.
Your task is to RESTRUCTURE them into a perfect study guide.

Here is the chaotic raw text from multiple pages:
{full_context}

RULES FOR RESTRUCTURING:
1. **Ignore Page Order**: Group facts by TOPIC, not by page number. If Page 1 and Page 8 discuss the same concept, merge them.
2. **Create Logical Headers**: Use # Main Topic and ## Subtopic.
3. **Fix Continuity**: If a sentence cuts off on one page and resumes on another, merge it.
4. **Format**: Use Markdown. Use bullet points for lists. Use code blocks for code.

Output the final study guide now.
<|end|>
<|assistant|>"""

        try:
            output = self.llm_pipe(
                prompt,
                max_new_tokens=2500,
                temperature=0.3,
                do_sample=True
            )

            final_notes = output[0]['generated_text']


            if "<|assistant|>" in final_notes:
                final_notes = final_notes.split("<|assistant|>")[1].strip()

            print("Notes structured successfully!")
            return final_notes

        except Exception as e:
            print(f"LLM error: {e}")
            # Fallback
            return f"""# Recovery Mode - Raw OCR Text

The LLM failed to structure the notes. Here is the raw extracted text:

{full_context}

*Note: This is unprocessed OCR output. It may contain errors and lacks structure.*"""

# Instantiating
digitizer = NotesDigitizer()

🚀 Using device: cuda
⏳ Loading Vision Model (OCR)...
⏳ Loading LLM (Standard FP16)...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Systems Online.


In [ ]:
# @title 3. Dashboard
from fastapi import FastAPI, File, UploadFile, Request, Response
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from pydantic import BaseModel
import uvicorn
import nest_asyncio
import os
import io
import markdown
from xhtml2pdf import pisa
from typing import List

app = FastAPI()

# temp dir
os.makedirs("templates", exist_ok=True)
templates = Jinja2Templates(directory="templates")

# pdf gen helper
def convert_markdown_to_pdf(markdown_text):
    html_body = markdown.markdown(markdown_text)
    full_html = f"""
    <html>
    <head>
        <style>
            body {{ font-family: Helvetica, sans-serif; font-size: 12pt; color: #333; }}
            h1 {{ color: #4f46e5; border-bottom: 2px solid #ddd; padding-bottom: 5px; }}
            h2 {{ color: #334155; margin-top: 20px; }}
            code {{ background-color: #f3f4f6; padding: 2px 5px; font-family: Courier; }}
            pre {{ background-color: #f1f5f9; padding: 10px; border-radius: 5px; }}
            ul {{ line-height: 1.6; }}
        </style>
    </head>
    <body>
        <div style="text-align: center; margin-bottom: 30px;">
            <h1 style="border:none;">Lecturify Notes</h1>
            <p style="color: #666; font-size: 10pt;">Generated by AI</p>
        </div>
        {html_body}
    </body>
    </html>
    """

    pdf_buffer = io.BytesIO()
    pisa_status = pisa.CreatePDF(full_html, dest=pdf_buffer)

    if pisa_status.err:
        return None
    return pdf_buffer.getvalue()

# --- HTML ---
html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Lecturify | AI Smart Notes</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css" rel="stylesheet">
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
    <style>
        :root { --primary: #4f46e5; --bg: #f3f4f6; }
        body { background-color: var(--bg); font-family: 'Segoe UI', sans-serif; }
        .navbar { background-color: white; box-shadow: 0 1px 3px rgba(0,0,0,0.1); }
        .brand-text { font-weight: 800; color: var(--primary); letter-spacing: -0.5px; }

        .upload-area { border: 2px dashed #cbd5e1; border-radius: 1rem; padding: 2rem; text-align: center; background: white; transition: all 0.2s; cursor: pointer; }
        .upload-area:hover { border-color: var(--primary); background: #eef2ff; }

        .preview-thumb { width: 60px; height: 60px; object-fit: cover; border-radius: 8px; margin-right: 10px; border: 1px solid #ddd; }

        .markdown-body { background: white; padding: 2rem; border-radius: 1rem; box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1); min-height: 300px; }
        /* Markdown Styles */
        .markdown-body h1 { border-bottom: 1px solid #eee; padding-bottom: 0.3em; color: #1e293b; }
        .markdown-body h2 { color: #334155; margin-top: 1.5em; }
        .markdown-body code { background: #f1f5f9; padding: 0.2em 0.4em; border-radius: 4px; color: #ef4444; }
        .markdown-body pre { background: #1e293b; color: #f8fafc; padding: 1em; border-radius: 8px; overflow-x: auto; }
    </style>
</head>
<body>

<nav class="navbar navbar-light mb-4 py-3">
    <div class="container">
        <span class="navbar-brand brand-text"><i class="fas fa-magic me-2"></i>Lecturify</span>
    </div>
</nav>

<div class="container pb-5">
    <div class="row g-4">
        <div class="col-lg-4">
            <div class="card shadow-sm border-0 h-100">
                <div class="card-body">
                    <h6 class="fw-bold text-muted mb-3">1. UPLOAD IMAGES</h6>

                    <div class="upload-area" onclick="document.getElementById('fileInput').click()">
                        <i class="fas fa-cloud-upload-alt fa-3x text-primary mb-3 opacity-50"></i>
                        <p class="mb-0 fw-bold">Click to select photos</p>
                        <small class="text-muted">Supports Batch Upload</small>
                        <input type="file" id="fileInput" multiple accept="image/*" class="d-none">
                    </div>

                    <div id="fileList" class="mt-3"></div>

                    <button id="convertBtn" class="btn btn-primary w-100 py-2 mt-4 fw-bold shadow-sm" disabled>
                        <i class="fas fa-bolt me-2"></i>DIGITIZE NOTES
                    </button>
                </div>
            </div>
        </div>

        <div class="col-lg-8">
            <div class="d-flex justify-content-between align-items-center mb-2">
                <h6 class="fw-bold text-muted mb-0">2. GENERATED NOTES</h6>
                <div>
                    <button class="btn btn-sm btn-danger me-2 d-none" id="pdfBtn" onclick="downloadPDF()"><i class="fas fa-file-pdf me-1"></i>PDF</button>
                    <button class="btn btn-sm btn-outline-secondary" onclick="copyToClipboard()"><i class="far fa-copy me-1"></i>Copy</button>
                </div>
            </div>

            <div id="loading" class="text-center py-5 d-none">
                <div class="spinner-border text-primary mb-3" role="status" style="width: 3rem; height: 3rem;"></div>
                <h5 class="text-dark">Processing with AI...</h5>
                <p class="text-muted small">Reading handwriting & structuring logic...</p>
            </div>

            <div id="resultArea" class="markdown-body">
                <div class="text-center text-muted py-5 opacity-50">
                    <i class="fas fa-book-open fa-3x mb-3"></i>
                    <p>Notes will appear here...</p>
                </div>
            </div>
        </div>
    </div>
</div>

<script>
    let selectedFiles = [];
    let currentNotes = "";

    // Handle File Selection
    document.getElementById('fileInput').onchange = (e) => {
        selectedFiles = Array.from(e.target.files);
        const listEl = document.getElementById('fileList');
        const btn = document.getElementById('convertBtn');

        listEl.innerHTML = '';

        if (selectedFiles.length > 0) {
            btn.disabled = false;
            selectedFiles.forEach(file => {
                const img = document.createElement('img');
                img.src = URL.createObjectURL(file);
                img.className = 'preview-thumb';
                listEl.appendChild(img);
            });
            const count = document.createElement('div');
            count.className = 'mt-2 small fw-bold text-success';
            count.innerText = selectedFiles.length + " images selected";
            listEl.appendChild(count);
        }
    };

    // Handle Conversion
    document.getElementById('convertBtn').onclick = async () => {
        const btn = document.getElementById('convertBtn');
        const loading = document.getElementById('loading');
        const resultArea = document.getElementById('resultArea');
        const pdfBtn = document.getElementById('pdfBtn');

        btn.disabled = true;
        resultArea.classList.add('d-none');
        pdfBtn.classList.add('d-none');
        loading.classList.remove('d-none');

        const formData = new FormData();
        selectedFiles.forEach(file => {
            formData.append("files", file);
        });

        try {
            const response = await fetch("/digitize-batch", {
                method: "POST",
                body: formData
            });
            const data = await response.json();

            currentNotes = data.notes; // Save for PDF export
            resultArea.innerHTML = marked.parse(data.notes);
            resultArea.classList.remove('d-none');
            pdfBtn.classList.remove('d-none'); // Show PDF button
        } catch (err) {
            alert("Error processing images.");
            console.error(err);
        } finally {
            loading.classList.add('d-none');
            btn.disabled = false;
        }
    };

    function copyToClipboard() {
        const text = document.getElementById('resultArea').innerText;
        navigator.clipboard.writeText(text);
        alert("Copied to clipboard!");
    }

    async function downloadPDF() {
        if(!currentNotes) return;

        const btn = document.getElementById('pdfBtn');
        btn.innerHTML = '<i class="fas fa-spinner fa-spin"></i> Generating...';

        try {
            const response = await fetch("/generate-pdf", {
                method: "POST",
                headers: { "Content-Type": "application/json" },
                body: JSON.stringify({ notes: currentNotes })
            });

            if (response.ok) {
                const blob = await response.blob();
                const url = window.URL.createObjectURL(blob);
                const a = document.createElement('a');
                a.href = url;
                a.download = "Lecturify_Notes.pdf";
                document.body.appendChild(a);
                a.click();
                document.body.removeChild(a);
            } else {
                alert("Failed to generate PDF");
            }
        } catch (e) {
            console.error(e);
            alert("Error downloading PDF");
        } finally {
            btn.innerHTML = '<i class="fas fa-file-pdf me-1"></i>PDF';
        }
    }
</script>
</body>
</html>
"""

with open("templates/index.html", "w") as f:
    f.write(html_code)

# api endpoints
class PDFRequest(BaseModel):
    notes: str

@app.get("/", response_class=HTMLResponse)
async def home(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

@app.post("/digitize-batch")
async def digitize_batch(files: List[UploadFile] = File(...)):
    file_bytes_list = []
    for file in files:
        content = await file.read()
        file_bytes_list.append(content)

    result_markdown = digitizer.process_batch(file_bytes_list)
    return {"notes": result_markdown}

@app.post("/generate-pdf")
async def generate_pdf(req: PDFRequest):
    pdf_bytes = convert_markdown_to_pdf(req.notes)
    if not pdf_bytes:
        return {"error": "PDF generation failed"}

    return Response(content=pdf_bytes, media_type="application/pdf")

print("Server ready.")

✅ Server ready.


In [ ]:
# @title 4. Going live
import threading
from pyngrok import ngrok
import uvicorn
import nest_asyncio


NGROK_AUTH_TOKEN = "....... get one for yourself"

ngrok.kill()
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

try:
    public_url = ngrok.connect(8000).public_url
    print(f"\n Public URL: {public_url}")
except Exception as e:
    print(f"Error starting tunnel: {e}")

def run_server():
    nest_asyncio.apply()
    uvicorn.run(app, port=8000, host="127.0.0.1", log_level="info")

thread = threading.Thread(target=run_server)
thread.start()


 Public URL: https://unattached-postrheumatic-zackary.ngrok-free.dev


INFO:     Started server process [3519]
